<a href="https://colab.research.google.com/github/dude123studios/AdvancedDeepLearning/blob/main/ConditionalGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
import numpy as np
from tensorflow.keras.datasets import mnist
from google.colab.patches import cv2_imshow

In [2]:
EPOCHS = 20
BATCH_SIZE = 100
NUM_CLASSES = 10
LATENT_DIM = 64

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/127.5-1, x_test/127.5-1
y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

In [4]:
def generator():
    z = Input(shape=(LATENT_DIM,), name='latent')
    c = Input(shape=(NUM_CLASSES,), name='conditional')
    y = Dense(64)(c)
    concat = Concatenate(axis=-1)([z, y])
    x = Dense(128*7*7)(concat)
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((7,7,128))(x)
    x = UpSampling2D()(x)

    x = Conv2D(128, kernel_size=3,padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    Conv2D(64, kernel_size=3,padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D()(x)

    x = Conv2D(32, kernel_size=3,padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(1, kernel_size=3,padding='same')(x)
    output = Activation('tanh')(x)
    return tf.keras.models.Model(inputs=(z, c), outputs=output)

In [5]:
def discriminator():
    model = tf.keras.models.Sequential()

    model.add(Conv2D(64,kernel_size=3,strides=2,padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.2))

    model.add(Conv2D(128,kernel_size=3,strides=2,padding='same'))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128,kernel_size=3,strides=2,padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.2))

    model.add(Conv2D(256,kernel_size=3,strides=2,padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    return model

In [6]:
class ConditionalGAN(tf.keras.Model):

    def __init__(self, **kwargs):
        super(ConditionalGAN, self).__init__(**kwargs)

        self.generator = generator()
        self.discriminator = discriminator()
        self.disc_head = Dense(1, activation='sigmoid')
        self.class_head = Dense(NUM_CLASSES, activation='softmax')
        self.beta = 0.5
    
    def compile(self, d_optimizer, g_optimizer, loss=None):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_metric = tf.keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = tf.keras.metrics.Mean(name="g_loss")
    
    def call(self, inputs):
        return self.generator({'latent':inputs[0], 'conditional':inputs[1]})

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]
    
    def train_step(self, data):
        
        self.beta += 2e-5

        real_imgs = data[0]
        class_labels = data[1]
        noise = tf.random.normal((BATCH_SIZE, LATENT_DIM))
        valid = tf.ones((BATCH_SIZE, 1))
        fake = tf.zeros((BATCH_SIZE, 1))

        with tf.GradientTape() as tape:
            fake_imgs = self([noise, class_labels])

            features_real = self.discriminator(real_imgs)
            features_fake = self.discriminator(fake_imgs)

            disc_real = self.disc_head(features_real)
            disc_fake = self.disc_head(features_fake)

            c_real = self.class_head(features_real)

            d_adversarial_loss = tf.keras.losses.binary_crossentropy(valid, disc_real)
            d_adversarial_loss += tf.keras.losses.binary_crossentropy(fake, disc_fake)
            d_class_loss = tf.keras.losses.categorical_crossentropy(class_labels, c_real)

            d_loss = d_adversarial_loss + self.beta * d_class_loss

        d_vars = self.discriminator.trainable_variables + self.class_head.trainable_variables + self.disc_head.trainable_variables
        d_grads = tape.gradient(d_loss, d_vars)
        self.d_optimizer.apply_gradients(zip(d_grads, d_vars))

        with tf.GradientTape() as tape:
            fake_imgs = self([noise, class_labels])

            features_fake = self.discriminator(fake_imgs)
            disc_fake = self.disc_head(features_fake)
            c_fake = self.class_head(features_fake)

            g_adversarial_loss = tf.keras.losses.binary_crossentropy(valid, disc_fake)
            g_class_loss = tf.keras.losses.categorical_crossentropy(class_labels, c_fake)

            g_loss = g_adversarial_loss + self.beta * g_class_loss
        
        g_vars = self.generator.trainable_variables
        g_grads = tape.gradient(g_loss, g_vars)
        self.g_optimizer.apply_gradients(zip(g_grads, g_vars))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)

        return {'d_loss':self.d_loss_metric.result(), 'g_loss':self.g_loss_metric.result()}

In [7]:
class ConditionalGANMonitor(tf.keras.callbacks.Callback):
    def __init__(self, num_img=5):
        self.num_img = num_img

    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1) % 5 is not 0:
            return 
        random_latent_vectors = tf.random.normal(shape=(self.num_img, LATENT_DIM))
        random_choice = np.random.randint(0, NUM_CLASSES, (self.num_img,))
        random_logit = tf.keras.utils.to_categorical(random_choice, NUM_CLASSES)
        generated_images = self.model([random_latent_vectors, random_logit])
        generated_images = (generated_images+1.)*127.5
        generated_images = generated_images.numpy()
        for i in range(self.num_img):
            img = generated_images[i]
            print('Number:', random_choice[i])
            cv2_imshow(img)

In [8]:
model = ConditionalGAN()
model.compile(tf.keras.optimizers.Adam(), tf.keras.optimizers.Adam())
callbacks = [ConditionalGANMonitor(5)]
model.fit(x_train,y_train, batch_size = BATCH_SIZE, epochs=EPOCHS, callbacks=callbacks)

Epoch 1/20
600/600 [==============================] - 27s 37ms/step - d_loss: 0.5385 - g_loss: 10.0939
Epoch 2/20
600/600 [==============================] - 22s 37ms/step - d_loss: 0.1223 - g_loss: 10.3360
Epoch 3/20
600/600 [==============================] - 22s 37ms/step - d_loss: 0.3465 - g_loss: 6.2233
Epoch 4/20
600/600 [==============================] - 22s 36ms/step - d_loss: 0.5997 - g_loss: 2.9385
Epoch 5/20
600/600 [==============================] - 22s 37ms/step - d_loss: 0.8121 - g_loss: 2.0454
Number: 6


Number: 5


Number: 3


Number: 5


Number: 6


...

Epoch 16/20
600/600 [==============================] - 22s 37ms/step - d_loss: 1.1501 - g_loss: 1.1900
Epoch 17/20
600/600 [==============================] - 22s 37ms/step - d_loss: 1.1853 - g_loss: 1.0889
Epoch 18/20
600/600 [==============================] - 22s 37ms/step - d_loss: 1.2075 - g_loss: 1.0514
Epoch 19/20
600/600 [==============================] - 22s 37ms/step - d_loss: 1.2116 - g_loss: 1.0456
Epoch 20/20
600/600 [==============================] - 22s 37ms/step - d_loss: 1.2329 - g_loss: 1.0067
Number: 8


Number: 3


Number: 2


Number: 6


Number: 0
